<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=97163d5c5491df78a84db5b07ea94cb8541ab5c7227401d58d2f01162a18d9f6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-07 09:30:01
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.41 C
-------------------
Today PnL: -1.44 L (-1.0%)
Current PnL: -23.09 L (-15.13%)
CY Booked + Current PnL: -10.44 L (-6.84%)
-------------------
Total profit:  1.58 L
Total loss:  -24.67 L
-------------------
Total Booked + Current PnL: 16.54 L (13.17%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.94%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 90.07 L (63.67%)
Deployed:  1.26 C
Current:  1.41 C
CAGR/XIRR %: 6.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.28,64.0,X-LC,6.45,232112.0,21746.0,7033.0,-0.55,10.34,3.03,13.68,37.0,3.09,1.65,28.57,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,100.43,50.0,M-SC,1.91,86355.0,-14422.0,14516.0,-0.63,-14.31,16.81,0.09,245.0,-0.99,0.61,13.45,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-6.98,60.0,M-LC,11.65,168935.0,28095.0,16404.0,-1.05,19.95,9.71,31.59,66.0,1.71,1.20,51.72,XY24,NTT,BANKS
2,ABBOTINDIA,35195.00,-10.45,42.0,X-MC,2.81,88365.0,-2313.0,17222.0,0.00,-2.55,19.49,16.44,101.0,-0.13,0.63,17.11,X40,ATH,PHARMA
33,ICICIGI,2252.93,-14.77,53.0,X-MC,5.77,146757.0,10624.0,19959.0,-0.89,7.80,13.60,22.47,91.0,0.53,1.05,21.80,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.0,-287.15,45.0,H-SC,11.22,104159.0,-59446.0,180758.0,2.78,-36.33,173.54,74.15,139.0,-0.33,0.74,33.51,XR,NTT,CHEMICALS
13,BERGEPAINT,680.0,-17.98,55.0,X-MC,1.26,220502.0,-6837.0,54905.0,1.30,-3.01,24.90,21.14,106.0,-0.12,1.57,24.29,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,88.62,33.0,M-SC,6.41,123554.0,-51005.0,51040.0,0.57,-29.22,41.31,0.02,221.0,-1.00,0.88,7.47,OX40N,NTT,PAINTS
74,TATAMOTORS,1065.0,-53.17,22.0,X-LC,32.34,148050.0,-128473.0,238553.0,0.33,-46.46,161.13,39.81,54.0,-0.54,1.05,4.35,XY24,NTT,AUTO
84,VOLTAS,1530.0,-7.37,29.0,X-MC,4.31,194430.0,2688.0,35075.0,0.29,1.40,18.04,19.69,99.0,0.08,1.38,8.29,XY25,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.00,57.89,44.0,H-SC,15.78,144922.0,-34149.0,99880.0,-4.73,-19.07,68.92,36.71,135.0,-0.34,1.03,43.79,XR,NTT,BANKS
71,SYMPHONY,1306.00,-44.75,43.0,M-SC,16.72,117762.0,-53379.0,53323.0,-4.24,-31.19,45.28,-0.03,196.0,-1.00,0.84,2.67,OX40N,NTT,DURABLES
1,ABB,7934.00,-42.20,36.0,H-LC,5.07,241632.0,-19987.0,139204.0,-3.71,-7.64,57.61,45.57,7.0,-0.14,1.72,2.82,AR,NTT,ELECTRICAL
14,BSOFT,836.99,-18.28,45.0,H-SC,10.99,91902.0,-48751.0,121531.0,-3.37,-34.66,132.24,51.74,131.0,-0.40,0.65,6.33,XR,ATH,IT
8,ATULAUTO,844.00,3229.73,28.0,M-SC,3.23,149498.0,-47634.0,137463.0,-3.29,-24.16,91.95,45.57,236.0,-0.35,1.06,5.99,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,5.57,51.0,H-SC,7.93,125748.0,451.0,58473.0,-2.3,0.36,46.5,47.03,141.0,0.01,0.9,57.45,MH,ATH,POWER


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,100.43,50.0,M-SC,1.91,86355.0,-14422.0,14516.0,-0.63,-14.31,16.81,0.09,245.0,-0.99,0.61,13.45,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.75,221310.0,-48357.0,84695.0,-0.92,-17.93,38.27,13.47,138.0,-0.57,1.58,12.53,XY24,NTT,PAINTS
17,CERA,9475.0,-24.11,42.0,H-SC,1.84,138230.0,-37673.0,79690.0,-2.82,-21.42,57.65,23.89,149.0,-0.47,0.98,19.38,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.50,97832.0,-29973.0,29956.0,-0.06,-23.45,30.62,-0.01,232.0,-1.00,0.70,10.78,OX40N,NTT,AC
67,SIS,528.0,1991.71,42.0,H-SC,2.53,84405.0,-26627.0,50238.0,-2.47,-23.98,59.52,21.26,156.0,-0.53,0.60,14.04,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,48.36,55.0,M-MC,10.21,233985.0,9023.0,158642.0,-1.47,4.01,67.80,74.53,192.0,0.06,1.67,37.56,XY24,BTT,STEEL
46,KPIGREEN,731.05,5.57,51.0,H-SC,7.93,125748.0,451.0,58473.0,-2.30,0.36,46.50,47.03,141.0,0.01,0.90,57.45,MH,ATH,POWER
79,UNIONBANK,163.00,-6.98,60.0,M-LC,11.65,168935.0,28095.0,16404.0,-1.05,19.95,9.71,31.59,66.0,1.71,1.20,51.72,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-40.66,51.0,M-SC,6.22,122930.0,8132.0,85903.0,-1.04,7.08,69.88,81.92,223.0,0.09,0.88,45.53,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.89,49.0,H-SC,5.39,128620.0,5284.0,126343.0,-1.87,4.28,98.23,106.72,119.0,0.04,0.92,28.66,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-40.66,51.0,M-SC,6.22,122930.0,8132.0,85903.0,-1.04,7.08,69.88,81.92,223.0,0.09,0.88,45.53,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.89,49.0,H-SC,5.39,128620.0,5284.0,126343.0,-1.87,4.28,98.23,106.72,119.0,0.04,0.92,28.66,AR,ATH,MISC
46,KPIGREEN,731.05,5.57,51.0,H-SC,7.93,125748.0,451.0,58473.0,-2.30,0.36,46.50,47.03,141.0,0.01,0.90,57.45,MH,ATH,POWER
31,HINDZINC,730.22,23.40,40.0,M-LC,9.67,201144.0,-3932.0,116503.0,-1.03,-1.92,57.92,54.89,52.0,-0.03,1.43,21.72,X5K,ATH,METALS
86,WIPRO,420.00,-16.52,39.0,M-LC,5.42,146940.0,-4005.0,113467.0,-1.27,-2.65,77.22,72.51,53.0,-0.04,1.05,3.21,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,32.34,148050.0,-128473.0,238553.0,0.33,-46.46,161.13,39.81,54.0,-0.54,1.05,4.35,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-42.01,24.0,X-SC,7.80,86752.0,-41918.0,87220.0,-0.52,-32.58,100.54,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
23,DMART,5391.45,-21.31,25.0,X-LC,7.54,200500.0,-9907.0,69072.0,-1.77,-4.71,34.45,28.12,38.0,-0.14,1.43,17.80,X40N,ATH,FMCG
44,JSWINFRA,342.00,-24.37,26.0,X-MC,4.54,189607.0,-10740.0,41240.0,-1.06,-5.36,21.75,15.23,178.0,-0.26,1.35,25.16,X40N,NTT,REALTY
84,VOLTAS,1530.00,-7.37,29.0,X-MC,4.31,194430.0,2688.0,35075.0,0.29,1.40,18.04,19.69,99.0,0.08,1.38,8.29,XY25,NTT,AC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-32.27,30.0,X-MC,0.15,195885.0,-4935.0,72732.0,-0.78,-2.46,37.13,33.76,80.0,-0.07,1.39,2.61,X40,ATH,FMCG
41,ITC,452.00,-38.34,47.0,X-LC,1.06,199732.0,-406.0,21291.0,0.23,-0.20,10.66,10.44,4.0,-0.02,1.42,5.35,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.39,37.0,H-LC,1.40,151930.0,-34165.0,81647.0,-1.46,-18.36,53.74,25.51,15.0,-0.42,1.08,11.96,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-9.58,32.0,X-MC,3.26,301770.0,-26221.0,132749.0,-0.37,-7.99,43.99,32.48,92.0,-0.20,2.15,2.52,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-7.24,61.0,X-LC,3.98,243036.0,8892.0,39736.0,-0.25,3.80,16.35,20.77,86.0,0.22,1.73,10.98,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-42.01,24.0,X-SC,7.80,86752.0,-41918.0,87220.0,-0.52,-32.58,100.54,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
53,QUESS,986.00,-50.96,31.0,X-SC,37.46,48452.0,-16554.0,167484.0,-0.84,-25.47,345.67,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-30.09,31.0,X-MC,7.44,78970.0,-4540.0,24568.0,-0.69,-5.44,31.11,23.98,82.0,-0.18,0.56,0.07,X40,ATH,APPARELS
20,COLPAL,3726.84,-7.40,38.0,X-MC,7.30,215950.0,-47415.0,156737.0,-0.65,-18.00,72.58,41.51,84.0,-0.30,1.54,1.34,XY25,ATH,FMCG
3,ACC,3906.00,-38.50,42.0,X-MC,2.38,182330.0,-55621.0,208276.0,-0.50,-23.37,114.23,64.15,174.0,-0.27,1.30,2.28,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.96,31.0,X-SC,37.46,48452.0,-16554.0,167484.0,-0.84,-25.47,345.67,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-30.09,31.0,X-MC,7.44,78970.0,-4540.0,24568.0,-0.69,-5.44,31.11,23.98,82.0,-0.18,0.56,0.07,X40,ATH,APPARELS
57,RELAXO,1176.00,-41.14,39.0,X-SC,4.24,80304.0,-64856.0,140781.0,-2.34,-44.68,175.31,52.31,136.0,-0.46,0.57,7.49,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-42.01,24.0,X-SC,7.80,86752.0,-41918.0,87220.0,-0.52,-32.58,100.54,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,42.0,X-MC,2.81,88365.0,-2313.0,17222.0,0.00,-2.55,19.49,16.44,101.0,-0.13,0.63,17.11,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.97,43.0,X-LC,9.73,279453.0,-66503.0,133215.0,-1.26,-19.22,47.67,19.28,1.0,-0.50,1.99,3.31,X40,ATH,IT
39,INFY,2275.00,-20.32,45.0,X-LC,5.00,310533.0,-2355.0,174054.0,-0.60,-0.75,56.05,54.87,3.0,-0.01,2.21,7.47,X40,BTT,IT
41,ITC,452.00,-38.34,47.0,X-LC,1.06,199732.0,-406.0,21291.0,0.23,-0.20,10.66,10.44,4.0,-0.02,1.42,5.35,X40,NTT,FMCG
83,VBL,671.64,-16.88,52.0,X-LC,6.34,299286.0,-16556.0,129232.0,-0.58,-5.24,43.18,35.67,5.0,-0.13,2.13,7.89,X40N,ATH,FMCG
1,ABB,7934.00,-42.20,36.0,H-LC,5.07,241632.0,-19987.0,139204.0,-3.71,-7.64,57.61,45.57,7.0,-0.14,1.72,2.82,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,34.0,L-SC,27.50,74246.0,-39303.0,79354.0,-1.38,-34.61,106.88,35.27,268.0,-0.50,0.53,81.72,XR,NTT,HOTELS
7,ASIANTILES,137.00,7394.44,65.0,L-SC,12.33,82228.0,-11582.0,88198.0,-1.43,-12.35,107.26,81.67,269.0,-0.13,0.59,59.24,XR,NTT,CERAMICS
49,MASFIN,398.61,-20.51,45.0,H-SC,8.31,90450.0,-7530.0,29134.0,-0.89,-7.69,32.21,22.05,152.0,-0.26,0.64,31.48,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.66,51.0,M-SC,6.22,122930.0,8132.0,85903.0,-1.04,7.08,69.88,81.92,223.0,0.09,0.88,45.53,XR,NTT,DURABLES
46,KPIGREEN,731.05,5.57,51.0,H-SC,7.93,125748.0,451.0,58473.0,-2.30,0.36,46.50,47.03,141.0,0.01,0.90,57.45,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,71.90,62.0,H-SC,3.94,148952.0,-33823.0,144886.0,-0.70,-18.51,97.27,60.77,125.0,-0.23,1.06,34.14,XR,NTT,JEWELLERY
68,SONACOMS,806.63,-32.59,62.0,M-SC,6.48,83816.0,-17343.0,57347.0,-0.82,-17.14,68.42,39.54,202.0,-0.30,0.60,18.19,AR,ATH,AUTO
46,KPIGREEN,731.05,5.57,51.0,H-SC,7.93,125748.0,451.0,58473.0,-2.30,0.36,46.50,47.03,141.0,0.01,0.90,57.45,MH,ATH,POWER
7,ASIANTILES,137.00,7394.44,65.0,L-SC,12.33,82228.0,-11582.0,88198.0,-1.43,-12.35,107.26,81.67,269.0,-0.13,0.59,59.24,XR,NTT,CERAMICS
62,SAMMAANCAP,326.00,-196.59,67.0,M-SC,33.95,166212.0,16992.0,127185.0,-0.33,11.39,76.52,96.62,208.0,0.13,1.18,79.88,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.71
1,25,44.38
2,50,75.44


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.72
LC    33.20
MC    23.09
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.93
X40      16.30
X40N     12.42
XY25     11.97
XR        9.88
AR        7.80
OX40N     6.55
X200      1.78
X5K       1.43
SR        1.05
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.19
X-LC    22.55
X-MC    18.77
M-SC    12.60
X-SC     5.48
M-LC     5.09
H-LC     4.58
H-MC     2.33
M-MC     1.67
L-SC     1.45
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.35,-4.99,38.94
IT,12.43,-20.53,85.11
FINANCE,12.10,-11.20,61.60
MISC,7.74,-19.55,79.85
ELECTRICAL,5.72,-12.40,52.25
PAINTS,5.66,-16.28,33.61
BANKS,4.00,-20.19,90.10
INSURANCE,3.87,-2.05,40.10
AUTO,2.71,-50.73,113.64


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3372680.0,22
XR,1268986.0,13
AR,1192435.0,9
X40,868510.0,12
X40N,723544.0,9
OX40N,564532.0,9
XY25,492700.0,8
SR,274307.0,2
X5K,116503.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3433258.0,24
M-SC,1276429.0,15
X-MC,1238207.0,14
X-LC,1111918.0,13
X-SC,579898.0,6
H-LC,295446.0,3
M-LC,294104.0,4
H-MC,261479.0,2
L-SC,258154.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1210822.0      6
           AR         846175.0      5
           XR         794577.0      7
M-SC       XY24       739720.0      6
X-MC       XY24       576078.0      4
X-LC       X40        471283.0      5
X-SC       X40N       325194.0      4
X-LC       XY24       318242.0      2
X-MC       X40        310007.0      6
H-SC       SR         274307.0      2
           OX40N      248904.0      3
X-LC       X40N       238040.0      3
H-LC       AR         220851.0      2
X-MC       XY25       191812.0      2
H-MC       XY24       185288.0      1
L-SC       XR         167552.0      2
X-SC       XY24       167484.0      1
X-MC       X40N       160310.0      2
M-MC       XY24       158642.0      1
M-SC       OX40N      148835.0      4
           XR         135280.0      2
           XY25       127185.0      1
           AR         125409.0      2
M-LC       X5K        116503.0      1
           XR         113467.0      1
L-SC       OX40N       90602.0      1
X-SC       X40         87220.0      1
X-LC       XY25        84353.0      3
H-MC       OX40N       76191.0      1
H-LC       X200        74595.0      1
H-SC       MH          58473.0      1
L-MC       XR          58110.0      1
M-LC       XY25        47730.0      1
L-LC       XY25        41620.0      1
M-LC       XY24        16404.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
